In [21]:
from openai import OpenAI
import os
import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
from tqdm import tqdm
import json, csv
load_dotenv('/Users/guida/llm_argument_tasks/.env')

api_key = os.environ.get('OPENAI_API_KEY')
client = OpenAI()

In [22]:
class ArgumentClassification(BaseModel):
    id: str = Field(description="The ID the comment being analyzed")    
    label: int = Field(description="The label associated with the argument (0 or 1)") 

In [23]:
# Dictionary for label-to-argument mappings for each topic
topic_label_to_argument = {
    "abortion": {
        "p-right": "Abortion is a woman’s right.",
        "p-rape": "Rape victims need it to be legal.",
        "p-not_human": "A fetus is not a human yet, so it's okay to abort.",
        "p-mother_danger": "Abortion should be allowed when a mother's life is in danger.",
        "p-baby_ill_treatment": "Unwanted babies are ill-treated by parents and/or not always adopted.",
        "p-birth_ctrl": "Birth control fails at times and abortion is one way to deal with it.",
        "p-not_murder": "Abortion is not murder.",
        "p-sick_mom": "Mother is not healthy/financially solvent.",
        "p-other": "Others",
        "c-adopt": "Put baby up for adoption.",
        "c-kill": "Abortion kills a life.",
        "c-baby_right": "An unborn baby is a human and has the right to live.",
        "c-sex": "Be willing to have the baby if you have sex.",
        "c-bad_4_mom": "Abortion is harmful for women.",
        "c-other": "Others"
    },
    "gayRights": {
        "p-normal": "Gay marriage is like any other marriage.",
        "p-right_denied": "Gay people should have the same rights as straight people.",
        "p-no_threat_for_child": "Gay parents can adopt and ensure a happy life for a baby.",
        "p-born": "People are born gay.",
        "p-religion": "Religion should not be used against gay rights.",
        "p-Other": "Others",
        "c-religion": "Religion does not permit gay marriages.",
        "c-abnormal": "Gay marriages are not normal/against nature.",
        "c-threat_to_child": "Gay parents cannot raise kids properly.",
        "c-gay_problems": "Gay people have problems and create social issues.",
        "c-Other": "Others"
    },
    "obama": {
        "p-economy": "Fixed the economy.",
        "p-War": "Ending the wars.",
        "p-republicans": "Better than the republican candidates.",
        "p-decision_policies": "Makes good decisions/policies.",
        "p-quality": "Has qualities of a good leader.",
        "p-health": "Ensured better healthcare.",
        "p-foreign_policies": "Executed effective foreign policies.",
        "p-job": "Created more jobs.",
        "p-Other": "Others",
        "c-economy": "Destroyed our economy.",
        "c-War": "Wars are still on.",
        "c-job": "Unemployment rate is high.",
        "c-health": "Healthcare bill is a failure.",
        "c-decision_policies": "Poor decision-maker.",
        "c-republicans": "We have better republicans than Obama.",
        "c-quality": "Not eligible as a leader.",
        "c-foreign_policies": "Ineffective foreign policies.",
        "c-Other": "Others"
    },
    "marijuana": {
        "p-not_addictive": "Not addictive.",
        "p-medicine": "Used as a medicine for its positive effects.",
        "p-legal": "Legalized marijuana can be controlled and regulated by the government.",
        "p-right": "Prohibition violates human rights.",
        "p-no_damage": "Does not cause any damage to our bodies.",
        "p-Other": "Others",
        "c-health": "Damages our bodies.",
        "c-mind": "Responsible for brain damage.",
        "c-illegal": "If legalized, people will use marijuana and other drugs more.",
        "c-crime": "Causes crime.",
        "c-addiction": "Highly addictive.",
        "c-Other": "Others"
    }
}

In [24]:
def classify_text_gpt(id: str, comment_text: str, topic: str, label: str) -> dict:
    label_to_argument = topic_label_to_argument.get(topic)
    argument = label_to_argument.get(label)
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment in relation to a specific argument about {topic}. You need to:
            Identify if the comment makes use of the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
        {"role": "user", "content": comment_text},
        ],
        response_format=ArgumentClassification,
    )

    return completion.choices[0].message.content

In [25]:
def process_dataframe_comments(df: pd.DataFrame, topic: str) -> List[dict]:
    results = [] 

    for idx, row in tqdm(df.iterrows(), desc="Processing comments", unit="comment", total=len(df)):
        comment_id = row['id'] 
        comment_text = row['text']  
        argument_text = row['label']
        
        try:
            classification = classify_text_gpt(
                id=comment_id, 
                comment_text=comment_text,  
                topic=topic, 
                label=argument_text 
            )

            results.append(classification) 

        except json.JSONDecodeError as e:
            print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
            continue

        except Exception as e:
            print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
            continue
        
    output_filename = f'yru_{topic}_identification_GPT.json'
    with open(output_filename, 'w') as f:
        json.dump(results, f, indent=2)

    return results

## Abortion

In [26]:
ab = pd.read_csv('../../clean_data/yru_abortion.csv')
topic = 'abortion'

process_dataframe_comments(ab, topic)

Processing comments:   0%|          | 1/739 [00:00<03:07,  3.94comment/s]

An unexpected error occurred for comment: Once again...  1. In all instances abortion is leg... - Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************yVgA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


Processing comments:   0%|          | 2/739 [00:00<02:58,  4.14comment/s]

An unexpected error occurred for comment: Once again...  1. In all instances abortion is leg... - Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************yVgA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


Processing comments:   0%|          | 3/739 [00:00<03:09,  3.88comment/s]

...

Processing comments:   1%|          | 7/739 [00:01<02:47,  4.38comment/s]

An unexpected error occurred for comment: Any Federal law on abortion is inherently illegal ... - Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************yVgA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


Processing comments:   1%|          | 8/739 [00:01<02:55,  4.17comment/s]

An unexpected error occurred for comment: The number of abortions are relatively small   Sor... - Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************yVgA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


KeyboardInterrupt: 

In [49]:
output_file = 'comarg_gm_argument_identification.json'
with open(output_file, 'w') as f:
    json.dump(classifications, f, indent=2)

## Under God in Pledge

In [57]:
ugip = pd.read_csv('../../clean_data/UGIP_structured.csv')

def process_comments_with_arguments(df: pd.DataFrame) -> List[dict]:
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
        comment_id = row['id'] 
        comment_text = row['comment_text']  
        argument_text = row['argument_text']

        try:
            classification = classify_text_gpt(comment_id, comment_text, argument_text)
            results.append(classification)

        except json.JSONDecodeError as e:
            print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
            continue

        except Exception as e:
            print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
            continue

    return results

classifications = process_comments_with_arguments(ugip)

output_file = 'comarg_ugip_argument_identification_gpt.json'
with open(output_file, 'w') as f:
    json.dump(classifications, f, indent=2)

Processing comments:   0%|          | 0/1013 [00:00<?, ?it/s]

Processing comments: 100%|██████████| 1013/1013 [12:57<00:00,  1.30it/s]


## YRU

In [ ]:
def classify_text(id: str, comment_text: str, topic: str, label: str) -> dict:
    label_to_argument = topic_label_to_argument.get(topic)
    argument = label_to_argument.get(label)

    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment in relation to a specific argument about {topic}. You need to:
            Identify if the comment makes use of the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": ArgumentClassification.model_json_schema(),
        }
    )
    
    return json.loads(extract.choices[0].message.content)

### Abortion

In [ ]:
def evaluate_predictions(golden_csv_path: str, predictions_json_path: str):
    """
    Evaluates predictions against the golden data.
    
    Parameters:
    - golden_csv_path: Path to the CSV file containing golden data.
    - predictions_json_path: Path to the JSON file containing predictions.
    
    Output:
    - Classification report comparing the predictions with the mapped gold labels.
    """

    golden_df = pd.read_csv(golden_csv_path)
    
    # Change labels to 1 for YRU data
    golden_df['label'] = 1 

    with open(predictions_json_path, 'r') as f:
        predictions = json.load(f)

    predictions_df = pd.DataFrame(predictions)
    
    # Ensure the IDs align between the golden data and the predictions
    merged_df = pd.merge(golden_df[['id', 'label']], predictions_df[['id', 'label']], on='id', suffixes=('_gold', '_pred'))

    # Map the gold labels to 0 and 1
    merged_df['label_gold'] = map_gold_labels(merged_df['label_gold'].astype(int))
    
    # Ground truth vector
    ground_truth = merged_df['label_gold'].values
    
    # Prediction vector
    predictions = merged_df['label_pred'].values

    report = classification_report(ground_truth, predictions)
    print(report)


## Evaluation GM

In [ ]:
golden_csv_path = '/Users/guida/llm_argument_tasks/clean_data/GM_structured.csv'
predictions_json_path = '/Users/guida/llm_argument_tasks/output_files/llama3/comarg_gm_argument_identification.json'

evaluate_predictions(golden_csv_path, predictions_json_path)

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       836
           1       0.61      0.52      0.56       431

    accuracy                           0.72      1267
   macro avg       0.69      0.67      0.68      1267
weighted avg       0.71      0.72      0.72      1267

